In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
root_paths = [
    "/data/kaggle-wikipedia/data/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

date_df.csv
key_1.csv
median_49.csv
page_df.csv
page_ohe.csv
sample_submission_1.csv
spider.txt
train_1.csv



In [3]:
train = pd.read_csv(root + 'train_1.csv')
train.fillna(0, inplace = True)

In [4]:
down_sample = None
if down_sample is not None:
    train = train[train.index % down_sample == 0]

In [5]:
class Heap:
    def __init__(self, isMinHeap):
        self.isMinHeap = isMinHeap
        self.data = []
        self.location = {}

    def isInOrder(self, a, b):
        if self.isMinHeap:
            return a <= b
        else:
            return a >= b

    def add(self, id, value):
        self.data.append((id, value))
        self.location[id] = self.getSize() - 1
        currentNode = self.data[-1]
        self.siftUp(currentNode)

    def remove(self, id):
        last = self.data[-1]
        if last[0] == id:
            self.data.pop()
            return
        i = self.location[id]
        del self.location[id]
        last = self.data.pop()
        self.data[i] = last
        self.location[last[0]] = i
        self.siftUp(last)
        self.siftDown(last)

    def getRoot(self):
        if len(self.data) == 0:
            return (-1, -sys.maxint)
        return self.data[0]

    def popRoot(self):
        if len(self.data) == 0:
            return None
        first = self.data[0]
        del self.location[first[0]]
        last = self.data.pop()
        if len(self.data) > 0:
            self.data[0] = last
            self.location[last[0]] = 0
            self.siftDown(last)
        return first


    def getSize(self):
        return len(self.data)

    def siftUp(self, x):
        id = x[0]
        value = x[1]
        i = self.location[x[0]]
        while i >=1 and self.isInOrder(self.data[i][1], self.data[int((i - 1)/2)][1]):
            parent_i = int((i - 1) / 2)
            temp = self.data[parent_i]
            self.data[parent_i] = self.data[i]
            self.data[i] = temp
            self.location[self.data[i][0]] = i
            self.location[self.data[parent_i][0]] = parent_i
            i = parent_i

    def siftDown(self, x):
        id = x[0]
        value = x[1]
        i = self.location[id]
        while (2*i + 1) < self.getSize():
            child_i = 2*i + 1
            child_i_2 = 2*i + 2
            if child_i_2 < self.getSize() and self.isInOrder(self.data[child_i_2][1], self.data[child_i][1]) :
                child_i = child_i_2
            if self.isInOrder(self.data[child_i][1], self.data[i][1]):
                temp = self.data[child_i]
                self.data[child_i] = self.data[i]
                self.data[i] = temp
                self.location[self.data[i][0]] = i
                self.location[self.data[child_i][0]] = child_i
            i = child_i

In [14]:
def medianSlidingWindow(nums, k, mask):
    """
        mask - 550 bool array
    """
    if nums is None or len(nums) == 0:
        return []
    n = len(nums)
    result = []
    maxHeap = Heap(False) #half smaller numbers 
    minHeap = Heap(True)


    for i in range(0, n):
        if mask[i]:
            first = i - k
            if first >= 0 and first in maxHeap.location:
                maxHeap.remove(first)
            elif first >= 0 and first in minHeap.location:
                minHeap.remove(first)
            if maxHeap.getSize() == 0 or nums[i] <= maxHeap.getRoot()[1]:
                maxHeap.add(i, nums[i])
            else:
                minHeap.add(i, nums[i])
            if maxHeap.getSize() - minHeap.getSize() >= 2:
                top = maxHeap.popRoot()
                minHeap.add(top[0], top[1])
            elif maxHeap.getSize() - minHeap.getSize() <= -1:
                top = minHeap.popRoot()
                maxHeap.add(top[0], top[1])
        if maxHeap.getSize() == 0:
            result.append(0)
        else:
            result.append(maxHeap.getRoot()[1])

    return result

In [18]:
median_df = train.copy()
train_np = train.iloc[:, 1:].values
median_np = median_df.iloc[:, 1:].values

In [19]:
window = 49

In [20]:
#mask
mask_name = 'weekday'
date_df = pd.read_csv(root + 'date_df.csv')
mask = date_df.isweekday.values == True

In [21]:
mask[:14]

array([ True,  True,  True, False, False,  True,  True,  True,  True,
        True, False, False,  True,  True], dtype=bool)

In [22]:
for i in range(len(median_df)):
# for i in range(1000):
    nums = train_np[i, :]
    cur_median = medianSlidingWindow(nums, window, mask)
    median_np[i, :] = cur_median
    if i % 1000 == 0:
        print('row {} is done'.format(i))
median_df.iloc[:, 1:] = median_np                            

row 0 is done
row 1000 is done
row 2000 is done
row 3000 is done
row 4000 is done
row 5000 is done
row 6000 is done
row 7000 is done
row 8000 is done
row 9000 is done
row 10000 is done
row 11000 is done
row 12000 is done
row 13000 is done
row 14000 is done
row 15000 is done
row 16000 is done
row 17000 is done
row 18000 is done
row 19000 is done
row 20000 is done
row 21000 is done
row 22000 is done
row 23000 is done
row 24000 is done
row 25000 is done
row 26000 is done
row 27000 is done
row 28000 is done
row 29000 is done
row 30000 is done
row 31000 is done
row 32000 is done
row 33000 is done
row 34000 is done
row 35000 is done
row 36000 is done
row 37000 is done
row 38000 is done
row 39000 is done
row 40000 is done
row 41000 is done
row 42000 is done
row 43000 is done
row 44000 is done
row 45000 is done
row 46000 is done
row 47000 is done
row 48000 is done
row 49000 is done
row 50000 is done
row 51000 is done
row 52000 is done
row 53000 is done
row 54000 is done
row 55000 is done
row 5

In [23]:
median_df.to_csv(root+'median_{}.csv'.format(mask_name), index = None)